In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "2").appName("Analysis").master("local[2]").getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

/home/sunbeam/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [21]:
%run "../includes/configuration"

In [29]:
w = spark.read.csv(r"/home/sunbeam/Desktop/FastLaneForecast/raw/weather.csv", header = True)
c = spark.read.csv(r"/home/sunbeam/Desktop/FastLaneForecast/raw/circuits.csv", header = True)
p = spark.read.parquet(f"{processed_folder_path}/pit_stops")
r = spark.read.parquet(f"{processed_folder_path}/results")

In [8]:
w.show(5)

+------+-----+------------+-------------------+------------+------------+-----------+-----------+--------------+
|season|round|  circuit_id|            weather|weather_warm|weather_cold|weather_dry|weather_wet|weather_cloudy|
+------+-----+------------+-------------------+------------+------------+-----------+-----------+--------------+
|  1950|    1| silverstone|   Sunny, Mild, Dry|           0|           0|          1|          0|             0|
|  1950|    2|      monaco|         Soleggiato|           1|           0|          0|          0|             0|
|  1950|    3|indianapolis|              Rainy|           0|           0|          0|          1|             0|
|  1950|    4|  bremgarten|Warm, dry and sunny|           1|           0|          1|          0|             0|
|  1950|    5|         spa|Warm, dry and sunny|           1|           0|          1|          0|             0|
+------+-----+------------+-------------------+------------+------------+-----------+-----------

In [9]:
c.show(5)

+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|circuitId| circuitRef|                name|    location|  country|     lat|    lng|alt|                 url|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|        1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|144.968| 10|http://en.wikiped...|
|        2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|101.738| 18|http://en.wikiped...|
|        3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|50.5106|  7|http://en.wikiped...|
|        4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|2.26111|109|http://en.wikiped...|
|        5|   istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517| 29.405|130|http://en.wikiped...|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
only showi

In [12]:
q = c.join(w, w["circuit_id"] == c["circuitRef"], "left")

In [30]:
r.show(5)

+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|       time|milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|      ingestion_date|
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+--------------------+
|        1|     18|        1|             1|    22|   1|       1|            1|             1|  10.0|  58|1:34:50.616|     5690616|         39|   2|        1:27.452|            218.3|2023-08-19 22:50:...|
|        2|     18|        2|             2|     3|   5|       2|            2|             2|   8.0|  58|     +5.478|     5696094|         41|   3|        1:27.739|          217.5

In [41]:
p.createOrReplaceTempView("pit_stops")
r.createOrReplaceTempView("results")

In [39]:
query = """
SELECT 
    driver_id,
    ROUND(AVG(milliseconds), 2) AS avg_pit_stops_time
FROM pit_stops
GROUP BY driver_id
ORDER BY AVG(milliseconds) ASC
"""

In [46]:
driver_id_time = spark.sql(query)

In [31]:
################################

In [47]:
driver_id_time.show()

+---------+------------------+
|driver_id|avg_pit_stops_time|
+---------+------------------+
|       30|          22541.64|
|        2|          22933.32|
|       17|          23556.55|
|      808|          23694.16|
|       10|          23743.95|
|      153|          23862.35|
|      811|          23938.56|
|      843|          23993.91|
|       67|          24024.32|
|      814|          24084.26|
|      823|          24441.02|
|        9|          24544.95|
|        5|          24608.67|
|       22|          24617.02|
|      845|          24706.52|
|      819|          24748.92|
|      816|          24838.52|
|       15|           25106.5|
|       37|          25247.86|
|      812|           25556.0|
+---------+------------------+
only showing top 20 rows



In [33]:
results = r.select("driver_id", "rank", "position")

In [34]:
results.show(5)

+---------+----+--------+
|driver_id|rank|position|
+---------+----+--------+
|        1|   2|       1|
|        2|   3|       2|
|        3|   5|       3|
|        4|   7|       4|
|        5|   1|       5|
+---------+----+--------+
only showing top 5 rows



In [67]:
new_df = results.join(driver_id_time, results.driver_id == driver_id_time.driver_id, "left")
new_df.show()

+---------+----+--------+---------+------------------+
|driver_id|rank|position|driver_id|avg_pit_stops_time|
+---------+----+--------+---------+------------------+
|        1|   2|       1|        1|          57788.47|
|        2|   3|       2|        2|          22933.32|
|        3|   5|       3|        3|          49760.69|
|        4|   7|       4|        4|          45973.43|
|        5|   1|       5|        5|          24608.67|
|        6|  14|       6|     null|              null|
|        7|  12|       7|     null|              null|
|        8|   4|       8|        8|          55434.48|
|        9|   9|    null|        9|          24544.95|
|       10|  13|    null|       10|          23743.95|
|       11|  15|    null|     null|              null|
|       12|  16|    null|     null|              null|
|       13|   6|    null|       13|          48945.79|
|       14|  11|    null|     null|              null|
|       15|  10|    null|       15|           25106.5|
|       16

In [70]:
new_df.select(new_df.columns[0], new_df.columns[1], new_df.columns[2], new_df.columns[4],).show()


AnalysisException: Reference 'driver_id' is ambiguous, could be: driver_id, pit_stops.driver_id.

In [50]:
new_df.createOrReplaceTempView("df")

In [56]:
query = """
SELECT 
    DISTINCT(driver_id), rank, position, avg_pit_stops_time
FROM df
ORDER BY avg_pit_stops_time ASC
"""

In [57]:
spark.sql(query).show()

AnalysisException: Reference 'driver_id' is ambiguous, could be: df.driver_id, df.driver_id.; line 3 pos 13